In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sns

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('vader_lexicon')

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Embedding

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
df=pd.read_table("tweet.csv")
df.head()

,tweet
0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,Finally a transparant silicon case ^^ Thanks t...
2,We love this! Would you go? #talk #makememorie...
3,I'm wired I know I'm George I was made that wa...
4,What amazing service! Apple won't even talk to...


In [3]:
df.isnull().sum()

tweet    0
dtype: int64

In [4]:
df.shape

(9873, 1)

In [5]:
def getsentiment(text):
    sid = SentimentIntensityAnalyzer()
    dic = sid.polarity_scores(text)
    
    value = dic["compound"]
    
    if(value>0):
        return "positive"
    else:
        return "negative"
        

In [6]:
df["label"] = df["tweet"].apply(getsentiment)
df.head()

,tweet,label
0,#fingerprint #Pregnancy Test https://goo.gl/h1...,negative
1,Finally a transparant silicon case ^^ Thanks t...,positive
2,We love this! Would you go? #talk #makememorie...,positive
3,I'm wired I know I'm George I was made that wa...,positive
4,What amazing service! Apple won't even talk to...,positive


In [7]:
def cleantext(text):
  token=word_tokenize(text.lower())
  ftoken=[t for t in token if (t.isalpha())]
  stopword= stopwords.words("english")
  ctoken= [t for t in ftoken if(t not in stopword)]
  lemma = WordNetLemmatizer()
  ltoken=[lemma.lemmatize(t) for t in ctoken]

  return " ".join(ltoken)

In [8]:
df.head()

,tweet,label
0,#fingerprint #Pregnancy Test https://goo.gl/h1...,negative
1,Finally a transparant silicon case ^^ Thanks t...,positive
2,We love this! Would you go? #talk #makememorie...,positive
3,I'm wired I know I'm George I was made that wa...,positive
4,What amazing service! Apple won't even talk to...,positive


In [9]:
df["label"].replace({"negative":0,"positive":1},inplace = True)

In [10]:
df["tweet"]=df["tweet"].apply(cleantext)

In [11]:
df.head()

,tweet,label
0,fingerprint pregnancy test http android apps b...,0
1,finally transparant silicon case thanks uncle ...,1
2,love would go talk makememories unplug relax i...,1
3,wired know george made way iphone cute daventr...,1
4,amazing service apple wo even talk question un...,1


In [12]:
x=df["tweet"]
y=df["label"]

In [13]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=1)

In [14]:
sentlen=[]
for sent in df["tweet"]:
  sentlen.append(len(word_tokenize(sent)))

df["sent_len"]=sentlen

In [15]:
df.head()

,tweet,label,sent_len
0,fingerprint pregnancy test http android apps b...,0,13
1,finally transparant silicon case thanks uncle ...,1,10
2,love would go talk makememories unplug relax i...,1,12
3,wired know george made way iphone cute daventr...,1,10
4,amazing service apple wo even talk question un...,1,11


In [16]:
max(sentlen)

33

In [17]:
np.quantile(sentlen,0.95)

20.0

In [18]:
max_len=100

In [19]:
df['sent_len'].max()

33

In [20]:
tok = Tokenizer(char_level=False, split=" ")
tok.fit_on_texts(xtrain)

In [21]:
tok.index_word

{1: 'http',
 2: 'iphone',
 3: 'apple',
 4: 'samsung',
 5: 'new',
 6: 'phone',
 7: 'sony',
 8: 'follow',
 9: 'ipad',
 10: 'love',
 11: 'like',
 12: 'life',
 13: 'android',
 14: 'rt',
 15: 'day',
 16: 'photo',
 17: 'instagram',
 18: 'get',
 19: 'cute',
 20: 'case',
 21: 'gain',
 22: 'today',
 23: 'got',
 24: 'fun',
 25: 'back',
 26: 'photography',
 27: 'galaxy',
 28: 'music',
 29: 'happy',
 30: 'news',
 31: 'app',
 32: 'time',
 33: 'io',
 34: 'work',
 35: 'girl',
 36: 'smile',
 37: 'one',
 38: 'funny',
 39: 'lol',
 40: 'make',
 41: 'beautiful',
 42: 'ipod',
 43: 'instagood',
 44: 'fashion',
 45: 'birthday',
 46: 'game',
 47: 'tech',
 48: 'itunes',
 49: 'iphonex',
 50: 'friend',
 51: 'everyone',
 52: 'photooftheday',
 53: 'amazing',
 54: 'gift',
 55: 'update',
 56: 'finally',
 57: 'apps',
 58: 'must',
 59: 'sougofollow',
 60: 'ca',
 61: 'u',
 62: 'follower',
 63: 'good',
 64: 'free',
 65: 'mac',
 66: 'would',
 67: 'selfie',
 68: 'want',
 69: 'rts',
 70: 'home',
 71: 'sale',
 72: 'baby',
 

In [22]:
vocab_len=len(tok.index_word)

In [23]:
len(tok.index_word)

13022

In [24]:
seq_train = tok.texts_to_sequences(xtrain)
seq_train

[[8, 238, 17, 1, 239, 233, 24, 215, 38, 113, 4],
 [4, 71, 460, 3212, 263, 767, 433, 2475, 4896, 1480],
 [4897, 1702, 4, 1159, 2003, 724, 129, 2476, 4898],
 [14, 4899, 10, 255, 5, 2004, 3213, 133, 46, 3214, 2477, 7, 327, 1064, 1],
 [103, 4900, 9, 369, 893, 348],
 [3215, 4, 4901, 1160, 1, 4902, 2478],
 [4903, 1289, 642, 894, 2005, 519, 2006, 226, 725],
 [821, 82, 3216, 138, 127, 3217, 535, 4904, 2479, 1703, 4905, 821, 3, 74],
 [66, 11, 120, 53, 15, 40, 94, 101, 196, 209, 22, 2, 125, 30, 12, 685],
 [3, 895, 3, 174, 87, 1161, 447, 114, 73, 3218, 4906, 1],
 [2, 302, 2480, 3, 302, 768, 1],
 [131, 769, 3219, 3, 104, 3220, 85, 3221, 2481, 1],
 [50, 448, 536, 4, 3, 4907, 34, 158, 1290, 1],
 [320, 227, 78, 139, 537, 2007, 60, 4908, 2008, 1481, 3, 4909],
 [3222, 370, 68, 40, 643, 1704, 2482, 16, 370, 3],
 [149, 42, 75, 3, 2483, 4910],
 [6, 2009, 121, 15, 686, 334, 80, 32, 75, 3],
 [35, 10, 3, 41, 770, 3, 419, 131, 4911, 591, 1],
 [1065, 2, 771, 2484, 1705, 434, 772, 2485, 1291, 55, 370, 3, 2486],

In [25]:
seq_mat_train = sequence.pad_sequences(seq_train,maxlen=max_len)
seq_mat_train

array([[    0,     0,     0, ...,    38,   113,     4],
       [    0,     0,     0, ...,  2475,  4896,  1480],
       [    0,     0,     0, ...,   129,  2476,  4898],
       ...,
       [    0,     0,     0, ...,   171, 13019,     1],
       [    0,     0,     0, ..., 13020,   700,     1],
       [    0,     0,     0, ...,  1562,  1444,     1]], dtype=int32)

In [26]:
rnn = Sequential()
rnn.add(Embedding(vocab_len+1,100,input_length=max_len,mask_zero=True))
rnn.add(SimpleRNN(units=100, activation="tanh"))

rnn.add(Dense(units=300, activation="relu"))
rnn.add(Dense(units=150, activation="relu"))
rnn.add(Dropout(0.2))
rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy",metrics="accuracy")
rnn.fit(seq_mat_train,ytrain,batch_size=30,epochs=8)





Epoch 1/8
231/231 [==============================] - 18s 70ms/step - loss: 0.5647 - accuracy: 0.7090
Epoch 2/8
231/231 [==============================] - 17s 72ms/step - loss: 0.2397 - accuracy: 0.9080
Epoch 3/8
231/231 [==============================] - 16s 71ms/step - loss: 0.0603 - accuracy: 0.9796
Epoch 4/8
231/231 [==============================] - 16s 71ms/step - loss: 0.0345 - accuracy: 0.9886
Epoch 5/8
231/231 [==============================] - 16s 70ms/step - loss: 0.0131 - accuracy: 0.9951
Epoch 6/8
231/231 [==============================] - 16s 71ms/step - loss: 0.0157 - accuracy: 0.9949
Epoch 7/8
231/231 [==============================] - 16s 71ms/step - loss: 0.0155 - accuracy: 0.9955
Epoch 8/8
231/231 [==============================] - 28s 122ms/step - loss: 0.0108 - accuracy: 0.9958


In [27]:
seq_mat_test=sequence.pad_sequences(tok.texts_to_sequences(xtest),maxlen=max_len)
ypred=rnn.predict(seq_mat_test)
ypred=ypred>0.5

from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.79      0.83      0.81      1804
           1       0.71      0.66      0.69      1158

    accuracy                           0.76      2962
   macro avg       0.75      0.75      0.75      2962
weighted avg       0.76      0.76      0.76      2962



In [28]:

ann = Sequential()
rnn.add(Embedding(vocab_len+1,100,input_length=max_len,mask_zero=True))
rnn.add(LSTM(units=100, activation="tanh"))

rnn.add(Dense(units=100, activation="relu"))
rnn.add(Dense(units=50, activation="relu"))
rnn.add(Dropout(0.2))
rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy",metrics="accuracy")
rnn.fit(seq_mat_train,ytrain,batch_size=30,epochs=50)

seq_mat_test=sequence.pad_sequences(tok.texts_to_sequences(xtest),maxlen=max_len)
ypred=rnn.predict(seq_mat_test)
ypred=ypred>0.5

from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

Epoch 1/50
231/231 [==============================] - 12s 31ms/step - loss: 0.4348 - accuracy: 0.8352
Epoch 2/50
231/231 [==============================] - 7s 31ms/step - loss: 0.3988 - accuracy: 0.8355
Epoch 3/50
231/231 [==============================] - 8s 36ms/step - loss: 0.4036 - accuracy: 0.8332
Epoch 4/50
231/231 [==============================] - 7s 31ms/step - loss: 0.3973 - accuracy: 0.8368
Epoch 5/50
231/231 [==============================] - 7s 31ms/step - loss: 0.4005 - accuracy: 0.8346
Epoch 6/50
231/231 [==============================] - 7s 31ms/step - loss: 0.3937 - accuracy: 0.8385
Epoch 7/50
231/231 [==============================] - 7s 32ms/step - loss: 0.3963 - accuracy: 0.8369
Epoch 8/50
231/231 [==============================] - 8s 35ms/step - loss: 0.3985 - accuracy: 0.8348
Epoch 9/50
231/231 [==============================] - 7s 32ms/step - loss: 0.3972 - accuracy: 0.8361
Epoch 10/50
231/231 [==============================] - 7s 31ms/step - loss: 0.3974 - accur